In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_community.utilities import SQLDatabase

sqlite_uri = 'sqlite:///Excel and databases/sameer.db'
db = SQLDatabase.from_uri(sqlite_uri)

In [5]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

sqlite
['UpdatedMergedEmployeeInsurance']

CREATE TABLE "UpdatedMergedEmployeeInsurance" (
	"Employee_Name" TEXT, 
	"EmpID" TEXT, 
	"MarriedID" TEXT, 
	"MaritalStatusID" TEXT, 
	"GenderID" TEXT, 
	"EmpStatusID" TEXT, 
	"DeptID" TEXT, 
	"PerfScoreID" TEXT, 
	"FromDiversityJobFairID" TEXT, 
	"Salary" REAL, 
	"Termd" TEXT, 
	"PositionID" TEXT, 
	"Position" TEXT, 
	"State" TEXT, 
	"Zip" TEXT, 
	"DOB" TEXT, 
	"Sex" TEXT, 
	"MaritalDesc" TEXT, 
	"CitizenDesc" TEXT, 
	"HispanicLatino" TEXT, 
	"RaceDesc" TEXT, 
	"DateofHire" TEXT, 
	"DateofTermination" TEXT, 
	"TermReason" TEXT, 
	"EmploymentStatus" TEXT, 
	"Department" TEXT, 
	"ManagerName" TEXT, 
	"ManagerID" TEXT, 
	"RecruitmentSource" TEXT, 
	"PerformanceScore" TEXT, 
	"EngagementSurvey" REAL, 
	"EmpSatisfaction" INTEGER, 
	"SpecialProjectsCount" INTEGER, 
	"LastPerformanceReview_Date" TEXT, 
	"DaysLateLast30" INTEGER, 
	"Absences" INTEGER, 
	"AnnualLeave" TEXT, 
	"SickLeave" TEXT, 
	"CasualLeave" TEXT, 
	"Insurance Plan Type" TEXT, 
	"Pre

In [29]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "who is sameer"})
# "what is price of `1968 Ford Mustang`"
print(query)

TypeError: create_sql_query_chain() got an unexpected keyword argument 'verbose'

In [19]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
print(execute_query.invoke(query))

Error: (sqlite3.OperationalError) near "[('M',)]": syntax error
[SQL: [('M',)]]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [20]:
chain = generate_query | execute_query
chain.invoke({"question": "How many absences does sameer has?"})

'[(9,)]'

In [21]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)

chain.invoke({"question": "Sameer is an employee or not?"})
     

'Yes, Sameer Khanzada is an employee.'

In [19]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate
# from langchain.memory import ConversationBufferMemory
# from langchain_core.runnables import RunnableLambda, RunnablePassthrough


# memory = ConversationBufferMemory(return_messages=True)

# def get_response(input, query, llm, memory):
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             ("human", f"{input}\nSQLQuery:"),
#             MessagesPlaceholder(variable_name="history"),
#             ("ai", f"{query}"),
#         ]
#     )
#     chain = (
#             RunnablePassthrough.assign(
#                 history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
#             )
#             | prompt
#             | llm
#         )
#     inputs = {"input": input}
#     response = chain.invoke(inputs)

#     # Save the context for future interactions
#     memory.save_context(inputs, {"output": response.content})
#     memory.load_memory_variables({})

#     return response.content

# input_message = "Who is sameer?"
# query = "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
# get_response(input_message, query, llm, memory)

"I'm sorry, I cannot execute SQL queries. How can I assist you with information about Sameer?"

In [23]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
    {
        "input": "Show product details for products in the 'Motorcycles' product line.",
        "query": "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    {
        "input": "Retrieve the names of employees who report to employee number 1002.",
        "query": "SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;"
    },
    {
        "input": "List all products with a stock quantity less than 7000.",
        "query": "SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;"
    },
    {
     'input':"what is price of `1968 Ford Mustang`",
     "query": "SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;"
    }
]

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=["input","top_k"],
    # input_variables=["input"],
)
print(few_shot_prompt.format(input1="is sameer an employee?"))
print("-----------------------------------------------------")

Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: Retrieve the names of employees who report to employee number 1002.
SQLQuery:
AI: SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: what is price of `1968 Ford Mustang`
SQLQuery:
AI: SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;
-----------------------------------------------------


In [27]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    vectorstore,
    k=2,
    input_keys=["input"],
)
example_selector.select_examples({"input": "how many employees we have?"})
# example_selector.select_examples({"input": "How many employees?"})

[{'input': 'Retrieve the names of employees who report to employee number 1002.',
  'query': 'SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;'},
 {'input': 'List all customers in France with a credit limit over 20,000.',
  'query': "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"}]

In [28]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input="How many products are there?"))

Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';


In [29]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?", table_info="some table info"))

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: How many products are there?


In [30]:
generate_query = create_sql_query_chain(llm, db,final_prompt)
chain = (
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "tell me attendence data of sameer"})

'Sameer Khanzada has been employed since April 24, 2024 in the Product Development department under the supervision of Jane Smith. As of now, Sameer has 9 absences.'

In [31]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info",messages=[]))

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: How many products are there?
